In [1]:
import time
from typing import Hashable
from functools import wraps


def benchmark(func):
    """
    ### Замер времени выполнения функции
    Декоратор, который замеряет время выполнения функции и выводит его в консоль.
    """

    @wraps(func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = func(*args, **kwargs)
        print(
            f"Function {func.__name__} took {time.time() - t1:.6f}",
            "seconds  to execute",
        )
        return result

    return wrapper


def logging(func):
    """
    ### Логирование параметров вызова функции
    Декоратор, который печатает параметры вызова функции.
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with {args}, {kwargs}")
        result = func(*args, **kwargs)
        return result

    return wrapper


def counter(func):
    """
    ### Счетчик вызовов функции
    Декоратор, который считает количество вызовов функции
    и выводит это количество в консоль.
    """
    n_times = 0

    @wraps(func)
    def wrapper(*args, **kwargs):
        nonlocal n_times
        n_times += 1
        print(f"Function {func.__name__} has been called for {n_times} times")
        return func(*args, **kwargs)

    return wrapper


def memo(func):
    """
    ### Кэширование ранее полученных результатов функции
    Если функция вызвана с теми же аргументами, что и ранее, то возвращается сохраненный результат.
    Используется для оптимизации вычислений, когда результаты могут быть повторно использованы.
    В примере ниже показано, как функция fibonacci может использовать кэширование для ускорения вычислений.
    ### Пример использования
    @memo
    def fibonacci(n: int) -> int:
        if n <= 1:
            return n
        return fibonacci(n - 1) + fibonacci(n - 2)
    """
    cache = {}

    @wraps(func)
    def wrapper(*args):
        is_hashable: bool = all(
            map(lambda arg: isinstance(arg, Hashable), args)
        )
        if is_hashable and (args in cache):
            print("Using cached result")
            return cache[args]
        result = func(*args)
        cache[args] = result
        return result

    return wrapper


@memo
@logging
@counter
@benchmark
def fibonacci(n: int) -> int:
    if n <= 1:
        return n
    return fibonacci(n - 1,) + fibonacci(n - 2,)


print(fibonacci(3))


Calling fibonacci with (3,), {}
Function fibonacci has been called for 1 times
Calling fibonacci with (2,), {}
Function fibonacci has been called for 2 times
Calling fibonacci with (1,), {}
Function fibonacci has been called for 3 times
Function fibonacci took 0.000001 seconds  to execute
Calling fibonacci with (0,), {}
Function fibonacci has been called for 4 times
Function fibonacci took 0.000002 seconds  to execute
Function fibonacci took 0.000166 seconds  to execute
Using cached result
Function fibonacci took 0.000275 seconds  to execute
2


In [3]:
import requests
import re
from random import randint


@counter
@logging
@benchmark
def word_count(word, url='https://www.gutenberg.org/files/2638/2638-0.txt'):
    
    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub(r'[\W]+' , ' ', raw).lower()
    
    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Function word_count has been called for 1 times
Calling word_count with ('whole',), {}
Function word_count took 4.570996 seconds  to execute
Cлово whole встречается 176 раз
